In [73]:
import yaml
from git import Repo, Git
import os
import shutil
from table_schema_to_markdown import convert_source
import frictionless
import glob
import json
import jsonschema
import re
from unidecode import unidecode
import codecs
import requests
from urllib import parse

In [74]:
CACHE_FOLDER = './cache'
DATA_FOLDER1 = './data'
DATA_FOLDER2 = './data2'
ERRORS_REPORT = []
SCHEMA_INFOS = {}
SCHEMA_CATALOG = {}

# Analyze validity of every release of every schema

In [75]:
# Loading yaml file containing all schemas that we want to display in schema.data.gouv.fr
with open("repertoires.yml", "r") as f:
    config = yaml.safe_load(f)

In [76]:
def clean_and_create_folder(folder):
    """Remove local folder if exist and (re)create it"""
    if os.path.exists(folder):
        shutil.rmtree(folder)
    os.mkdir(folder)

In [77]:
def get_consolidated_version(tag):
    """Analyze tag from a code source release, cast it to acceptable semver version X.X.X"""
    valid_version = True
    # Removing 'v' or 'V' from tag
    version_items = str(tag).replace('v','').replace('V','').split('.')
    # Add a patch number if only 2 items
    if len(version_items) == 2: version_items.append('0')
    # If more than 3, do not accept tag
    if len(version_items) > 3: valid_version = False
    # Verify if all items are digits
    for v in version_items:
        if not v.isdigit():
            valid_version = False
    # Return semver version and validity of it
    return '.'.join(version_items), valid_version

In [78]:
def manage_errors(repertoire_slug, version, reason):
    """Create dictionnary that will populate ERRORS_REPORT object"""
    mydict = {}
    mydict['schema'] = repertoire_slug
    mydict['version'] = version
    mydict['type'] = reason
    ERRORS_REPORT.append(mydict)

In [79]:
def check_schema(repertoire_slug, conf, schema_type):
    """Check validity of schema and all of its releases"""
    # schema_name in schema.data.gouv.fr is referenced by group and repo name in Git. Ex : etalab / schema-irve
    schema_name = '/'.join(conf['url'].split('.git')[0].split('/')[-2:])
    # define source folder and create it
    # source folder will help us to checkout to every release and analyze source code for each one
    src_folder = CACHE_FOLDER + '/' + schema_name + '/'
    os.makedirs(src_folder)
    # clone repo in source folder
    Repo.clone_from(conf['url'], src_folder)
    repo = Repo(src_folder)
    # get tags of repo
    tags = sorted(repo.tags, key=lambda t: t.commit.committed_datetime)
    
    list_schemas = {}
    
    # Defining SCHEMA_INFOS object for website use
    SCHEMA_INFOS[schema_name] = {}
    SCHEMA_INFOS[schema_name]['homepage'] = conf['url']
    SCHEMA_INFOS[schema_name]['external_doc'] = conf['external_doc'] if 'external_doc' in conf else None
    SCHEMA_INFOS[schema_name]['external_tool'] = conf['external_tool'] if 'external_tool' in conf else None
    SCHEMA_INFOS[schema_name]['type'] = conf['type']
    SCHEMA_INFOS[schema_name]['email'] = conf['email']
    SCHEMA_INFOS[schema_name]['versions'] = {}
    
    # for every tags
    for t in tags:
        # get semver version and validity of it
        version, valid_version = get_consolidated_version(t)
        # if semver ok
        if(valid_version):
            # define destination folder and create it
            # destination folder will store pertinents files for website for each version of each schema
            dest_folder = DATA_FOLDER1 + '/' + schema_name + '/' + version + '/'
            os.makedirs(dest_folder)
            # checkout to current version
            g = Git(src_folder)
            g.checkout(str(t))
            
            conf_schema = None
            # Managing validation differently for each type of schema
            # tableschema will use frictionless package
            # jsonschema will use jsonschema package
            # other will only check if schema.yml file is present and contain correct information
            if(schema_type == 'tableschema'):
                list_schemas = manage_tableschema(src_folder, dest_folder, list_schemas, version, schema_name)
            if(schema_type == 'jsonschema'):
                list_schemas, conf_schema = manage_jsonschema(src_folder, dest_folder, list_schemas, version, schema_name)
            if(schema_type == 'other'):
                list_schemas, conf_schema = manage_other(src_folder, dest_folder, list_schemas, version, schema_name)
    
    # Find latest valid version and create a specific folder 'latest' copying files in it (for website use)
    latest_folder, list_schemas, schema_file = manage_latest_folder(conf, list_schemas, schema_name)
    # Complete catalog with all relevant information of schema in it
    schema_to_add_to_catalog = generate_catalog_object(latest_folder, list_schemas, schema_file, schema_type, schema_name, conf_schema)
    return schema_to_add_to_catalog

In [80]:
def manage_tableschema(src_folder, dest_folder, list_schemas, version, schema_name):
    """Check validity of a schema release from tableschema type"""
    # Verify that a file schema.json is present
    if(os.path.isfile(src_folder + 'schema.json')):
        # Validate it with frictionless package
        frictionless_report = frictionless.validate(src_folder + 'schema.json')
        # If schema release is valid, then
        if(frictionless_report['valid'] == True):
            list_schemas[version] = 'schema.json'
            # We complete info of version
            SCHEMA_INFOS[schema_name]['versions'][version] = {}
            SCHEMA_INFOS[schema_name]['versions'][version]['pages'] = []
            # We check for list of normalized files if it is present in source code, if so, we copy paste them into dest folder
            for f in ['schema.json', 'README.md', 'SEE_ALSO.md', 'CHANGELOG.md', 'CONTEXT.md']:
                if(os.path.isfile(src_folder + f)):
                    shutil.copyfile(src_folder + f,dest_folder + f)
                    # if it is a markdown file, we will read them as page in website
                    if(f[-3:] == '.md'):
                        SCHEMA_INFOS[schema_name]['versions'][version]['pages'].append(f)
                    # if it is the schema, we indicate it as it in object
                    if(f == 'schema.json'):
                        SCHEMA_INFOS[schema_name]['versions'][version]['schema_url'] = '/' + schema_name + '/' + version + '/' + 'schema.json'
            # Create documentation file and save it
            with open(dest_folder + 'documentation.md', "w") as out:
                try:
                    # From schema.json, we use tableschema_to_markdown package to convert it in a
                    # readable mardown file that will be use for documentation
                    convert_source(dest_folder + 'schema.json', out, 'page',[])
                    SCHEMA_INFOS[schema_name]['versions'][version]['pages'].append('documentation.md')
                except:
                    # if conversion is on error, we add it to ERRORS_REPORT
                    manage_errors(repertoire_slug, version, 'convert to markdown')
        # If schema release is not valid, we remove it from DATA_FOLDER1
        else:
            manage_errors(repertoire_slug, version, 'tableschema validation')
            shutil.rmtree(dest_folder)
    # If there is no schema.json, schema release is not valid, we remove it from DATA_FOLDER1
    else:
        manage_errors(repertoire_slug, version, 'missing schema.json')
        shutil.rmtree(dest_folder)
    
    return list_schemas

In [81]:
def manage_jsonschema(src_folder, dest_folder, list_schemas, version, schema_name):
    """Check validity of a schema release from jsonschema type"""
    conf_schema = None
    # Verify that a file schemas.yml is present
    # This file will indicate title, description of jsonschema (it is a prerequisite asked by schema.data.gouv.fr)
    # This file will also indicate which file store the jsonschema schema
    if(os.path.isfile(src_folder + 'schemas.yml')):
        try:
            with open(src_folder + 'schemas.yml', "r") as f:
                conf_schema = yaml.safe_load(f)
                if('schemas' in conf_schema):
                    s = conf_schema['schemas'][0]
                    # Verify if jsonschema file indicate in schemas.yml is present, then load it
                    if(os.path.isfile(src_folder + s['path'])):
                        with open(src_folder + s['path'], "r") as f:
                            schema_data = json.load(f)
                        # Validate schema with jsonschema package
                        jsonschema.validators.validator_for(schema_data).check_schema(schema_data)
                        list_schemas[version] = s['path']
                        # We complete info of version
                        SCHEMA_INFOS[schema_name]['versions'][version] = {}
                        SCHEMA_INFOS[schema_name]['versions'][version]['pages'] = []
                        # We check for list of normalized files if it is present in source code, if so, we copy paste them into dest folder
                        for f in ['README.md', 'SEE_ALSO.md', 'CHANGELOG.md', 'CONTEXT.md', s['path']]:
                            if(os.path.isfile(src_folder + f)):
                                shutil.copyfile(src_folder + f,dest_folder + f)
                            # if it is a markdown file, we will read them as page in website
                            if(f[-3:] == '.md'):
                                SCHEMA_INFOS[schema_name]['versions'][version]['pages'].append(f)
                            # if it is the schema, we indicate it as it in object
                            if(f == s['path']):
                                SCHEMA_INFOS[schema_name]['versions'][version]['schema_url'] = '/' + schema_name + '/' + version + '/' + s['path']
        # If schema release is not valid, we remove it from DATA_FOLDER1
        except:
            manage_errors(repertoire_slug, version, 'jsonschema validation')
            shutil.rmtree(dest_folder)
    # If there is no schemas.yml, schema release is not valid, we remove it from DATA_FOLDER1    
    else:
        manage_errors(repertoire_slug, version, 'missing schemas.yml')
        shutil.rmtree(dest_folder)
    
    return list_schemas, conf_schema
     

In [82]:
def manage_other(src_folder, dest_folder, list_schemas, version, schema_name):
    """Check validity of a schema release from other type"""
    conf_schema = None
    # Verify that a file schema.yml is present
    # This file will indicate title, description of schema (it is a prerequisite asked by schema.data.gouv.fr)
    if(os.path.isfile(src_folder + 'schema.yml')):
        try:
            with open(src_folder + 'schema.yml', "r") as f:
                conf_schema = yaml.safe_load(f)
            list_schemas[version] = 'schema.yml'
            # We complete info of version
            SCHEMA_INFOS[schema_name]['versions'][version] = {}
            SCHEMA_INFOS[schema_name]['versions'][version]['pages'] = []
            # We check for list of normalized files if it is present in source code, if so, we copy paste them into dest folder
            for f in ['README.md', 'SEE_ALSO.md', 'CHANGELOG.md', 'CONTEXT.md', 'schema.yml']:
                if(os.path.isfile(src_folder + f)):
                    shutil.copyfile(src_folder + f,dest_folder + f)
                    # if it is a markdown file, we will read them as page in website
                    if(f[-3:] == '.md'):
                        SCHEMA_INFOS[schema_name]['versions'][version]['pages'].append(f)
                    # if it is the schema, we indicate it as it in object
                    if(f == 'schema.yml'):
                        SCHEMA_INFOS[schema_name]['versions'][version]['schema_url'] = '/' + schema_name + '/' + version + '/' + 'schema.yml'
        # If schema release is not valid, we remove it from DATA_FOLDER1
        except:
            manage_errors(repertoire_slug, version, 'validation of type other')
            shutil.rmtree(dest_folder)
    # If there is no schema.yml, schema release is not valid, we remove it from DATA_FOLDER1    
    else:
        manage_errors(repertoire_slug, version, 'missing schema.yml')
        shutil.rmtree(dest_folder)
    
    return list_schemas, conf_schema
     

In [83]:
def manage_latest_folder(conf, list_schemas, schema_name):
    """Create latest folder containing all files from latest valid version of a schema"""
    # Get all valid version from a schema by analyzing folders, then sort them to get latest valid version and related folder
    subfolders = [ f.name for f in os.scandir(DATA_FOLDER1 + '/' + schema_name + '/') if f.is_dir() ]
    subfolders.sort()
    latest_version_folder = DATA_FOLDER1 + '/' + schema_name + '/' + subfolders[-1] + '/'
    # Determine latest folder path then mkdir it
    latest_folder = DATA_FOLDER1 + '/' + schema_name + '/latest/'
    os.makedirs(latest_folder)
    # For every file in latest valid version folder, copy them into 
    for f in glob.glob(latest_version_folder + '*'):
        shutil.copyfile(f,latest_folder + f.split('/')[-1])
    # For website need, copy paste latest README into root of schema folder
    shutil.copyfile(latest_version_folder+'README.md','/'.join(latest_version_folder.split('/')[:-2])+'/README.md')
    # Indicate in schema_info object name of latest schema
    SCHEMA_INFOS[schema_name]['latest'] = subfolders[-1]
    return latest_folder, list_schemas, list_schemas[subfolders[-1]]
    

In [84]:
def generate_catalog_object(latest_folder, list_schemas, schema_file, schema_type, schema_name, obj_info=None):
    """Generate dictionnary containing all relevant information for catalog"""
    # If tableschema, relevant information are directly into schema.json, 
    # if not, relevant info are in yaml files with are stored in obj_info variable
    if(schema_type == 'tableschema'):
        with open(latest_folder + schema_file, "r") as f:
            schema = json.load(f)
    else:
        schema = obj_info
    # Complete dictionnary with relevant info needed in catalog
    mydict = {}
    mydict['name'] = schema_name
    mydict['title'] = schema['title']
    mydict['description'] = schema['description']
    mydict['schema_url'] = 'https://schema.data.gouv.fr/schemas/' + schema_name + '/latest/' + schema_file
    mydict['schema_type'] = schema_type
    mydict['contact'] = conf['email']
    mydict['examples'] = schema['resources'] if 'resources' in schema else []
    mydict['versions'] = []
    for sf in list_schemas:
        mydict2 = {}
        mydict2['version_name'] = sf
        mydict2['schema_url'] = 'https://schema.data.gouv.fr/schemas/' + schema_name + '/' + sf + '/' + list_schemas[sf]
        mydict['versions'].append(mydict2)
    # These three following property are not in catalog spec 
    mydict['external_doc'] = conf['external_doc'] if 'external_doc' in conf else None
    mydict['external_tool'] = conf['external_tool'] if 'external_tool' in conf else None
    mydict['homepage'] = conf['url']
    return mydict

In [86]:
# Clean and (re)create CACHE AND DATA FOLDER
clean_and_create_folder(CACHE_FOLDER)
clean_and_create_folder(DATA_FOLDER1)

# Initiate Catalog
SCHEMA_CATALOG['$schema'] = 'https://opendataschema.frama.io/catalog/schema-catalog.json'
SCHEMA_CATALOG['version'] = 1
SCHEMA_CATALOG['schemas'] = []

# For every schema in repertoires.yml, check it
for repertoire_slug, conf in config.items():
    schema_to_add_to_catalog = check_schema(repertoire_slug, conf, conf['type'])
    # Append info to SCHEMA_CATALOG
    SCHEMA_CATALOG['schemas'].append(schema_to_add_to_catalog)
    print('--- {} processed'.format(repertoire_slug))

# Save catalog to schemas.json file
with open(DATA_FOLDER1 + '/schemas.json', 'w') as fp:
    json.dump(SCHEMA_CATALOG, fp)
    
# Save schemas_infos to schema-infos.json file
with open(DATA_FOLDER1 + '/schema-infos.json', 'w') as fp:
    json.dump(SCHEMA_INFOS, fp)
    
# Save errors to errors.json file
with open(DATA_FOLDER1 + '/errors.json', 'w') as fp:
    json.dump(SCHEMA_INFOS, fp)

--- irve processed
--- format-commande-publique processed
--- decp-dpa processed
--- catalogue processed
--- deliberations processed
--- equipements processed
--- subventions processed
--- covoiturage processed
--- infos-travaux processed
--- stationnement processed
--- budget processed
--- dae processed
--- prenoms processed
--- arbres_urbains processed
--- amenagements-cyclables processed
--- stationnement-cyclable processed
--- inclusion-numerique processed
--- bal processed
--- hautes-remunerations processed
--- menus-restauration processed
--- plats-restauration processed
--- accessibilite-erp processed
--- sdirve processed
--- registre-archive processed
--- datatourisme processed
--- schema-zfe processed
--- arrete-circulation-marchandise processed
--- stations-taxi processed
--- vehicules-faibles-emissions-renouvellement-parc processed
--- point-eau-incendie processed
--- itineraire-randonnee processed
--- emprise-stationnement-voirie processed
--- passage-a-niveau processed
---

# Preparation for website

## schema relative files

In [87]:
def find_md_links(md):
    """Returns dict of links in markdown:
    'regular': [foo](some.url)
    'footnotes': [foo][3]
    
    [3]: some.url
    """
    # https://stackoverflow.com/a/30738268/2755116
    INLINE_LINK_RE = re.compile(r'\[([^\]]+)\]\(([^)]+)\)')
    FOOTNOTE_LINK_TEXT_RE = re.compile(r'\[([^\]]+)\]\[(\d+)\]')
    FOOTNOTE_LINK_URL_RE = re.compile(r'\[(\d+)\]:\s+(\S+)')

    links = list(INLINE_LINK_RE.findall(md))
    footnote_links = dict(FOOTNOTE_LINK_TEXT_RE.findall(md))
    footnote_urls = dict(FOOTNOTE_LINK_URL_RE.findall(md))

    footnotes_linking = []
        
    for key in footnote_links.keys():
        footnotes_linking.append((footnote_links[key], footnote_urls[footnote_links[key]]))

    return links

In [88]:
def cleanLinksDocumentation(dest_folder):
    """Custom cleaning for links in markdown"""
    # For every documentation.md file, do some custom cleaning for links
    file = codecs.open(dest_folder + 'documentation.md', "r", "utf-8")
    data = file.read()
    file.close()
    # Find all links in file
    links = find_md_links(data)
    # For each one, lower string then manage space ; _ ; --- and replace them by -
    for (name, link) in links:
        if(link.startswith('#')):
            newlink = link.lower()
            newlink = newlink.replace(' ','-')
            newlink = newlink.replace('_','-')
            newlink = unidecode(newlink, "utf-8")
            newlink = newlink.replace('---','-')
            data = data.replace(link,newlink)
    # Save modifications
    with open(dest_folder + 'documentation.md', 'w', encoding='utf-8') as fin:
        fin.write(data)

In [89]:
def addFrontToMarkdown(dest_folder, f):
    """Custom add to every markdown files"""
    # for every markdown files
    file = codecs.open(dest_folder + f, "r", "utf-8")
    data = file.read()
    file.close()
    # Add specific tag for website interpretation
    data = "<MenuSchema />\n\n"+data
    # Exception scdl Budget not well interpreted by vuepress
    data = data.replace('<DocumentBudgetaire>', 'DocumentBudgetaire')
    # Save modification
    with open(dest_folder + f, 'w', encoding='utf-8') as fin:
        fin.write(data)

In [90]:
def getListOfFiles(dirName):
    """Get list off all files in a specific folder"""
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
    return allFiles
                

In [91]:
# Get list of all files in DATA_FOLDER
files = []
files = getListOfFiles(DATA_FOLDER1)
# Create list of file that we do not want to copy paste
avoid_files = [DATA_FOLDER1 + '/' + s['name'] + '/README.md' for s in SCHEMA_CATALOG['schemas']]
# for every file
for f in files:
    # if it is a markdown, add custom front to content
    if(f[-3:] == '.md'):
        addFrontToMarkdown('/'.join(f.split('/')[:-1])+'/', f.split('/')[-1])
    # if it is the documentation file, clean links on it
    if(f.split('/')[-1] == 'documentation.md'):
        cleanLinksDocumentation('/'.join(f.split('/')[:-1])+'/')
    # if it is a README file (except if on avoid_list), copy paste it to root folder of schema (for website use)
    # That will create README file with name X.X.X.md (X.X.X corresponding to a specific version)
    if(f.split('/')[-1] == 'README.md'):
        if f not in avoid_files:
            shutil.copyfile(f, f.replace('/README.md','.md'))

# Clean and (re)create DATA_FOLDER2, then copy paste all DATA_FOLDER1 into DATA_FOLDER2
# DATA_FOLDER1 will be use to contain all markdown files
# DATA_FOLDER2 will be use to contain all yaml and json files 
# This is needed for vuepress that need to store page in one place and 'resources' in another
if os.path.exists(DATA_FOLDER2):
    shutil.rmtree(DATA_FOLDER2)
shutil.copytree(DATA_FOLDER1, DATA_FOLDER2)

'./data2'

## stats file

In [92]:
def get_contributors(url):
    """Get list off all contributors of a specific git repo"""
    parse_url = parse.urlsplit(url)
    # if github, use github api
    if('github.com' in parse_url.netloc):
        api_url =  parse_url.scheme+'://api.github.com/repos/'+parse_url.path[1:].replace('.git','')+'/contributors'
    # else, use gitlab api
    else:
        api_url =  parse_url.scheme+'://'+parse_url.netloc+'/api/v4/projects/'+parse_url.path[1:].replace('/','%2F').replace('.git','')+'/repository/contributors'
    try:
        r = requests.get(api_url)
        return len(r.json())
    except:
        return None

In [93]:

# For every issue, request them by label schema status (en investigation or en construction)
mydict = {}
labels = ['construction', 'investigation']
# For each label, get relevant info via github api of schema.data.gouv.fr repo
for l in labels:
    r = requests.get('https://api.github.com/repos/etalab/schema.data.gouv.fr/issues?q=is%3Aopen+is%3Aissue&labels=Sch%C3%A9ma%20en%20'+l)
    mydict[l] = []
    for issue in r.json():
        mydict2 = {}
        mydict2['created_at'] = issue['created_at']
        mydict2['labels'] = [l]
        mydict2['nb_comments'] = issue['comments']
        mydict2['title'] = issue['title']
        mydict2['url'] = issue['html_url']
        mydict[l].append(mydict2)

# Find number of current issue in schema.data.gouv.fr repo
r = requests.get('https://api.github.com/repos/etalab/schema.data.gouv.fr/issues?q=is%3Aopen+is%3Aissue')
mydict['nb_issues'] = len(r.json())

# for every schema, find relevant info in data.gouv.fr API
mydict['references'] = {}
for s in SCHEMA_CATALOG['schemas']:
    r = requests.get('https://www.data.gouv.fr/api/1/datasets/?schema='+s['name'])
    mydict['references'][s['name']] = {}
    mydict['references'][s['name']]['dgv_resources'] = r.json()['total']
    mydict['references'][s['name']]['title'] = s['title']
    mydict['references'][s['name']]['contributors'] = get_contributors(s['homepage'])

# Save stats infos to stats.json file
with open(DATA_FOLDER2 + '/stats.json', 'w') as fp:
    json.dump(mydict, fp)


## Prepare prod folders for website

In [94]:
def remove_all_files_extension(folder, extension):
    """Remove all file of a specific extension in a folder"""
    files = []
    files = getListOfFiles(folder)
    for f in files:
        if f[-1*len(extension):] == extension:
            os.remove(f)

In [95]:
# Remove all markdown from DATA_FOLDER1 and all json, yaml and yml file of DATA_FOLDER2
remove_all_files_extension(DATA_FOLDER2, '.md')
remove_all_files_extension(DATA_FOLDER1, '.json')
remove_all_files_extension(DATA_FOLDER1, '.yml')
remove_all_files_extension(DATA_FOLDER1, '.yaml')

# Remove actual currend prod folder in website
folder_to_remove = glob.glob('../site/site/*/')
folder_to_remove.append('../site/site/.vuepress/public/schemas')
for folder in folder_to_remove:
    if os.path.exists(folder):
        shutil.rmtree(folder)

# Copy all markdown in site folder, all resources in site/.vuepress/public/schemas/folder
shutil.copytree(DATA_FOLDER1, '../site/site/', dirs_exist_ok=True)
shutil.copytree(DATA_FOLDER2, '../site/site/.vuepress/public/schemas/', dirs_exist_ok=True)

# Copy three main json files into another public folder
shutil.copyfile('../site/site/.vuepress/public/schemas/schemas.json','../site/site/.vuepress/public/schemas.json')
shutil.copyfile('../site/site/.vuepress/public/schemas/stats.json','../site/site/.vuepress/public/stats.json')
shutil.copyfile('../site/site/.vuepress/public/schemas/schema-infos.json','../site/site/.vuepress/public/schema-infos.json')
shutil.copyfile('../site/site/.vuepress/public/schemas/errors.json','../site/site/.vuepress/public/errors.json')


'../site/site/.vuepress/public/schema-infos.json'